In [3]:
from custom_eval import eval_video
import os

MAX_BATCHES = 113

def checkpoint_time(checkpoint: str):
    checkpoint = checkpoint.split('-')
    epoch = int(checkpoint[0][5:])
    batch = int(checkpoint[1][5:-3])
    return epoch, batch

def normalize_batch(epoch, batch):
    return epoch + (batch / MAX_BATCHES)

def get_checkpoints(run_name: str):
    weights_dir = os.path.join('runs', 'detect', run_name, 'weights')
    checkpoints = sorted(
        [c for c in os.listdir(weights_dir) if 'batch' in c],
        key=lambda x: normalize_batch(*checkpoint_time(x))
    )
    return weights_dir, checkpoints

In [4]:
from ultralytics import YOLO
from custom_eval import eval_video, get_test_files, BINARY_KEYS
import json
TEST_DIR = "datasets/04_Test_Datasets/"

test_files = get_test_files(TEST_DIR)

def eval_checkpoints(run_name):
    weights_dir, checkpoints = get_checkpoints(run_name)

    for checkpoint in checkpoints:
        epoch, batch = checkpoint_time(checkpoint)
        print(f"EVAL ON EPOCH {epoch}, BATCH {batch}")
        model = YOLO(os.path.join(weights_dir, checkpoint))
        res = eval_video(model, test_files, 1e-3, 0.6, BINARY_KEYS)
        res['epoch'] = epoch
        res['batch'] = batch
        with open(os.path.join('runs', 'detect', run_name, 'bias.jsonl'), 'a') as f:
            f.write(json.dumps(res) + "\n")



In [5]:
eval_checkpoints('train21')

EVAL ON EPOCH 0, BATCH 0
starting eval...


  0%|          | 0/81 [00:00<?, ?it/s]

<class 'ultralytics.models.yolo.detect.predict.DetectionPredictor'>


/home/coinse/Documents/lm/spring2024/CS570-Video-Classification/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:84.)
  return F.conv2d(input, weight, bias, self.stride,
/home/coinse/Documents/lm/spring2024/CS570-Video-Classification/.venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
100%|██████████| 81/81 [03:03<00:00,  2.27s/it]


NUM CORRECT: 21 / 81 (0.26)
LABELS DISTRIBUTION:
{'None': 12, 'bird': 8, 'plane': 61}
PREDICTIONS DISTRIBUTION:
{'bird': {'False_Neg': 4,
          'False_Pos': 53,
          'True_Neg': 20,
          'True_Pos': 4,
          'guess_rate': 0.7037037037037037},
 'plane': {'False_Neg': 44,
           'False_Pos': 7,
           'True_Neg': 13,
           'True_Pos': 17,
           'guess_rate': 0.2962962962962963}}
EVAL ON EPOCH 0, BATCH 10
starting eval...


...

NUM CORRECT: 52 / 81 (0.64)
LABELS DISTRIBUTION:
{'None': 12, 'bird': 8, 'plane': 61}
PREDICTIONS DISTRIBUTION:
{'bird': {'False_Neg': 1,
          'False_Pos': 27,
          'True_Neg': 46,
          'True_Pos': 7,
          'guess_rate': 0.41975308641975306},
 'plane': {'False_Neg': 16,
           'False_Pos': 2,
           'True_Neg': 18,
           'True_Pos': 45,
           'guess_rate': 0.5802469135802469}}
EVAL ON EPOCH 2, BATCH 113
starting eval...


  0%|          | 0/81 [00:00<?, ?it/s]

<class 'ultralytics.models.yolo.detect.predict.DetectionPredictor'>


100%|██████████| 81/81 [02:57<00:00,  2.19s/it]

NUM CORRECT: 54 / 81 (0.67)
LABELS DISTRIBUTION:
{'None': 12, 'bird': 8, 'plane': 61}
PREDICTIONS DISTRIBUTION:
{'None': {'False_Neg': 11,
          'False_Pos': 0,
          'True_Neg': 69,
          'True_Pos': 1,
          'guess_rate': 0.012345679012345678},
 'bird': {'False_Neg': 1,
          'False_Pos': 24,
          'True_Neg': 49,
          'True_Pos': 7,
          'guess_rate': 0.38271604938271603},
 'plane': {'False_Neg': 15,
           'False_Pos': 3,
           'True_Neg': 17,
           'True_Pos': 46,
           'guess_rate': 0.6049382716049383}}
